In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio

# Running the exploration on the full DB used in our model

In [ ]:
import pandas as pd

# Read the original file
df_paris = pd.read_csv("model_paris_20_24.csv")

In [ ]:
# Basic stats
print("Number of rows : {}".format(df_paris.shape[0]))
print("Number of columns : {}".format(df_paris.shape[1]))
print()
print("Percentage of missing values: ")
display(100 * df_paris.isnull().sum() / df_paris.shape[0])
print()

In [ ]:
df_paris.dtypes

## Analyse surface

In [ ]:
# Visualisation des outliers surface_reelle_bati avec Plotly

# Box plot pour identifier visuellement les outliers
fig = px.box(
    df_paris, 
    y='surface_reelle_bati',
    title="Distribution des surfaces plotbox"
)
fig.update_layout(
    yaxis_title="Surface réelle bâti (m²)"
)
fig.show()

In [ ]:
# Histogramme 
fig = px.histogram(
    df_paris, 
    x='surface_reelle_bati',
    nbins=50,
    title="Distribution des surfaces réelles bâties (histogramme)"
)
fig.update_layout(
    xaxis_title="Surface réelle bâti (m²)",
    yaxis_title="Nombre de transactions"
)
fig.show()

## Analyse distribution prix_m2

In [ ]:
# Box plot pour identifier visuellement les outliers
fig = px.box(
    df_paris, 
    y='prix_m2',
    title="Distribution des prix au m²"
)
fig.update_layout(
    yaxis_title="Prix au m² (€)"
)
fig.show()

# Histogramme pour voir la distribution
fig = px.histogram(
    df_paris, 
    x='prix_m2',
    nbins=50,
    title="Distribution des prix au m²"
)
fig.update_layout(
    xaxis_title="Prix au m² (€)",
    yaxis_title="Nombre de transactions"
)
fig.show()

## Analye nombre de pièces 

In [ ]:
print(df_paris['nombre_pieces_principales'].describe())


In [ ]:
# 4. Box plot pour identifier les outliers
fig = px.box(
    df_paris,
    y='nombre_pieces_principales',
    title="Distribution nombre de pièces"
)
fig.update_layout(
    yaxis_title="Nombre de pièces principales"
)
fig.show()

## Analyse géographique

In [ ]:
df_paris['arrondissement'].value_counts().sort_index()

In [ ]:
# 2. Prix moyen par arrondissement
prix_par_arrond = df_paris.groupby('arrondissement')['prix_m2'].agg(['mean', 'median', 'count']).round(0)
prix_par_arrond.columns = ['Prix_moyen', 'Prix_median', 'Nb_transactions']
prix_par_arrond = prix_par_arrond.sort_values('Prix_moyen', ascending=False)

print("Prix par arrondissement (classé par prix moyen):")
print(prix_par_arrond)
print()

In [ ]:
fig = px.bar(
    x=prix_par_arrond.index,
    y=prix_par_arrond['Prix_moyen'],
    title="Prix moyen au m² par arrondissement"
)
fig.update_layout(
    xaxis_title="Arrondissement",
    yaxis_title="Prix moyen (€/m²)"
)
fig.show()

In [ ]:
# 4. Scatter plot géographique avec prix
fig = px.scatter(
    df_paris,
    x='longitude',
    y='latitude',
    color='prix_m2',
    title="Répartition géographique des prix au m²",
    color_continuous_scale='Viridis'
)
fig.update_layout(
    xaxis_title="Longitude",
    yaxis_title="Latitude"
)
fig.show()

## Property impact analysis 

In [ ]:
prix_par_pieces = df_paris.groupby('nombre_pieces_principales')['prix_m2'].agg(['mean', 'median', 'count']).round(0)
prix_par_pieces.columns = ['Prix_moyen', 'Prix_median', 'Nb_transactions']

print("1. Prix par nombre de pièces:")
print(prix_par_pieces)
print()

fig = px.box(
    df_paris,
    x='nombre_pieces_principales',
    y='prix_m2',
    title="Distribution des prix au m² par nombre de pièces"
)
fig.update_layout(
    xaxis_title="Nombre de pièces principales",
    yaxis_title="Prix au m² (€)"
)
fig.show()

In [ ]:
# 3. Impact des lots multiples
lots_impact = df_paris.groupby('nombre_lots')['prix_m2'].agg(['mean', 'median', 'count']).round(0)
lots_impact.columns = ['Prix_moyen', 'Prix_median', 'Nb_transactions']

print("2. Impact des lots multiples:")
print(lots_impact)
print()

fig = px.box(
    df_paris,
    x='nombre_lots',
    y='prix_m2',
    title="Impact des lots multiples sur le prix au m²"
)
fig.update_layout(
    xaxis_title="Nombre de lots",
    yaxis_title="Prix au m² (€)"
)
fig.show()

## Correlation

In [ ]:
numeric_features = [
    'prix_m2', 'surface_reelle_bati', 
    'nombre_pieces_principales', 'longitude', 'latitude', 
    'nombre_lots','arrondissement','annee'
]

# Calcul de la matrice de corrélation
corr_matrix = df_paris[numeric_features].corr()

In [ ]:
# Heatmap de corrélation complète
fig = px.imshow(
    corr_matrix,
    text_auto=True,
    aspect="auto",
    title="Matrice de corrélation - Toutes les features"
)
fig.update_layout(
    width=800,
    height=600
)
fig.show()

In [ ]:
#  Visualisation des corrélations avec prix_m2
correlations_prix = corr_matrix['prix_m2'].sort_values(ascending=False)

fig = px.bar(
    x=correlations_prix.drop('prix_m2').values,
    y=correlations_prix.drop('prix_m2').index,
    orientation='h',
    title="Corrélations avec prix_m2 (target variable)"
)
fig.update_layout(
    xaxis_title="Corrélation",
    yaxis_title="Features"
)
fig.show()


# FAIRE UNIQUEMENT AVEC LES DONNEES NUMERIQUES